In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os,sys
import pickle
import pandas as pd

2023-04-05 11:41:56.350149: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-05 11:41:56.351281: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-05 11:41:56.375016: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-05 11:41:56.375545: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 11:41:56.848333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
# Storing path of files
hlaPATH = "./Scrapper/hla_msa.txt"
afterPcaPATH = './after_pca.txt'
epitopePATH = './remove0123_sample100.csv'

# Data pre-processing

In [3]:
# loading the after_pca file which contains 12 most relevant physicochemical properties after PCA
after_pca = np.loadtxt(afterPcaPATH)

In [4]:
after_pca.shape

(21, 12)

In [5]:
# reading the epitope data
ori = pd.read_csv(epitopePATH)

In [6]:
# sampling the data and re-indexing the same
ori = ori.sample(frac=1, replace=False, random_state = 4).set_index(pd.Index(np.arange(ori.shape[0])))
ori.head()

,peptide,HLA,immunogenicity,test,respond,potential
0,DYIIKIWKL,HLA-A*2402,Negative,5,0,0.267811
1,VPVVEDALM,HLA-B*3501,Positive,0,0,0.969088
2,MMQDREDQSI,HLA-A*0201,Negative,24,0,0.108715
3,VEMTEPYNL,HLA-B*4402,Negative,4,0,0.294861
4,FIFSYVVAV,HLA-A*0201,Positive,0,0,0.973914


In [ ]:
ori.shape

In [ ]:
# reading HLA data
df = pd.read_csv(hlaPATH,header=None,delim_whitespace=True)
df.columns = ["HLA","pseudo"]

In [ ]:
df

In [ ]:
# making dictionary of HLA data by mapping HLA with its pseudo-sequence
hla_dic = {row["HLA"]:row["pseudo"] for index, row in df.iterrows()}

In [ ]:
hla_dic["HLA-A*0101"]

In [ ]:
# storing the HLA enteries in a list
inventory = list(hla_dic.keys())

In [ ]:
inventory[0]

In [ ]:
# generating dic_inventory for mapping the HLA sequences according to their types
dic_inventory = {}
for hla in inventory:
    if hla[4] in dic_inventory:
        if hla[6:8] in dic_inventory[hla[4]]:
            dic_inventory[hla[4]][hla[6:8]].append(hla[8:])
        else:
            dic_inventory[hla[4]][hla[6:8]] =[]
            dic_inventory[hla[4]][hla[6:8]].append(hla[8:])
    else:
        dic_inventory[hla[4]] = {}
        dic_inventory[hla[4]][hla[6:8]] =[]
        dic_inventory[hla[4]][hla[6:8]].append(hla[8:])

In [ ]:
# Functions for encoding the HLA and peptide using aaindex and mapping the same to immunogenic potential,taken 
# directly from deepimmuno, for maintaining uniformity of the input data

def rescue_unknown_hla(hla, dic_inventory):
    type_ = hla[4]
    first2 = hla[6:8]
    last2 = hla[8:]
    big_category = dic_inventory[type_]
    #print(hla)
    if not big_category.get(first2) == None:
        small_category = big_category.get(first2)
        distance = [abs(int(last2) - int(i)) for i in small_category]
        optimal = min(zip(small_category, distance), key=lambda x: x[1])[0]
        return 'HLA-' + str(type_) + '*' + str(first2) + str(optimal)
    else:
        small_category = list(big_category.keys())
        distance = [abs(int(first2) - int(i)) for i in small_category]
        optimal = min(zip(small_category, distance), key=lambda x: x[1])[0]
        return 'HLA-' + str(type_) + '*' + str(optimal) + str(big_category[optimal][0])

def aaindex(peptide,after_pca):

    amino = 'ARNDCQEGHILKMFPSTWYV-'
    matrix = np.transpose(after_pca)   # [12,21]
    encoded = np.empty([len(peptide), 12])  # (seq_len,12)
    for i in range(len(peptide)):
        query = peptide[i]
        if query == 'X': query = '-'
        query = query.upper()
        encoded[i, :] = matrix[:, amino.index(query)]

    return encoded

def hla_data_aaindex(hla_dic,hla_type,after_pca,dic_inventory):    # return numpy array [34,12,1]
    try:
        seq = hla_dic[hla_type]
    except KeyError:
        hla_type = rescue_unknown_hla(hla_type,dic_inventory)
        seq = hla_dic[hla_type]
    encode = aaindex(seq,after_pca)
    encode = encode.reshape(encode.shape[0], encode.shape[1], -1)
    return encode

def peptide_data_aaindex(peptide,after_pca):   # return numpy array [10,12,1]
    length = len(peptide)
    if length == 10:
        encode = aaindex(peptide,after_pca)
    elif length == 9:
        peptide = peptide[:5] + '-' + peptide[5:]
        encode = aaindex(peptide,after_pca)
    encode = encode.reshape(encode.shape[0], encode.shape[1], -1)
    return encode


def construct_aaindex(ori,hla_dic,after_pca,dic_inventory):
    series = []
#     res = pd.DataFrame(columns = ['encode_pep', 'encode_hla', 'immuno'])
    for i in range(ori.shape[0]):
        peptide = ori['peptide'].iloc[i]
        hla_type = ori['HLA'].iloc[i]
        immuno = np.array(ori['potential'].iloc[i]).reshape(1,-1)   # [1,1]

        encode_pep = peptide_data_aaindex(peptide,after_pca)    # [10,12]

        encode_hla = hla_data_aaindex(hla_dic,hla_type,after_pca,dic_inventory)   # [64,12]
#         res.loc[i] = [encode_pep.reshape(10,12).astype(float), encode_hla.reshape(64,12).astype(float), immuno]
        series.append((encode_pep, encode_hla, immuno))
#     return res
    return series

In [ ]:
# generating the final database
dataset = construct_aaindex(ori, hla_dic, after_pca,dic_inventory)

In [ ]:
# separating input1, input2 and labels while preserving the indexing
input1 = np.empty([len(dataset),10,12,1])
input2 = np.empty([len(dataset),64,12,1])
label = np.empty([len(dataset),1])

for i in range(len(dataset)):
    input1[i,:,:,:] = dataset[i][0]
    input2[i,:,:,:] = dataset[i][1]
    label[i,:] = dataset[i][2]


In [ ]:
# splitting the data into training (80%) and validation (20%) 
array = np.arange(len(dataset))
train_index = np.random.choice(array,int(len(dataset)*0.8),replace=False)
valid_index = [item for item in array if item not in train_index]
input1_train = input1[train_index]
input1_val = input1[valid_index]
input2_train = input2[train_index]
input2_val = input2[valid_index]
label_train = label[train_index]
label_val = label[valid_index]

# Model 1 - CNN 
(After modifying kernel of deepImmuno to match updated data)

In [ ]:
def buildCNN():
    input_pep = keras.Input(shape=(10, 12, 1))
    input_mhc = keras.Input(shape=(64, 12, 1))

    x = layers.Conv2D(filters=16, kernel_size=(2, 12))(input_pep)  # 9
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(filters=32, kernel_size=(2, 1))(x)    # 8
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPool2D(pool_size=(2, 1), strides=(2, 1))(x)  # 4
    x = layers.Flatten()(x)
    x = keras.Model(inputs=input_pep, outputs=x)

    y = layers.Conv2D(filters=16, kernel_size=(33, 12))(input_mhc)     # 32
    y = layers.BatchNormalization()(y)
    y = keras.activations.relu(y)
    y = layers.MaxPool2D(pool_size=(2, 1), strides=(2, 1))(y)  # 16
    y = layers.Conv2D(filters=32,kernel_size=(9,1))(y)    # 8
    y = layers.BatchNormalization()(y)
    y = keras.activations.relu(y)
    y = layers.MaxPool2D(pool_size=(2, 1),strides=(2,1))(y)  # 4
    y = layers.Flatten()(y)
    y = keras.Model(inputs=input_mhc,outputs=y)

    combined = layers.concatenate([x.output,y.output])
    z = layers.Dense(128,activation='relu')(combined)
    z = layers.Dropout(0.2)(z)
    z = layers.Dense(1,activation='sigmoid')(z)

    model = keras.Model(inputs=[input_pep,input_mhc],outputs=z)
    return model

In [ ]:
cnn_model = buildCNN()
cnn_model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    )

callback_val = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=False)
callback_train = keras.callbacks.EarlyStopping(monitor='loss',patience=2,restore_best_weights=False)

history = cnn_model.fit(
    x=[input1_train,input2_train],   # feed a list into
    y=label_train,
    validation_data = ([input1_val,input2_val],label_val),
    batch_size=128,
    epochs=200,
    callbacks = [callback_val,callback_train]
    )

In [ ]:
# plotting validation and training loss against number of epochs
import matplotlib.pyplot as plt
plt.plot()
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()

In [ ]:
# predicting and further evaluating binary classification accuracy
predictions = cnn_model.predict([input1_val,input2_val])
exact_match = 0
for i in range(len(predictions)):
    if predictions[i]>0.5 and label_val[i]>0.5:
        exact_match+=1
    elif predictions[i]<=0.5 and label_val[i]<=0.5:
        exact_match+=1
    else:
        continue
print("Accuracy : ",exact_match/len(predictions))

In [ ]:
# Dumping model to pickle file
pickle.dump(cnn_model, open('cnn_model.pkl', 'wb'))

# Model 2 - ANN - Fully connected

In [ ]:
def buildANN():
    input_pep = keras.Input(shape=(10, 12, 1))
    input_mhc = keras.Input(shape=(64, 12, 1))

    x = layers.Flatten()(input_pep)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dense(64,activation='relu')(x)
    x = layers.Dense(32,activation='relu')(x)
    x = keras.Model(inputs=input_pep, outputs=x)

    y = layers.Flatten()(input_mhc)
    y = layers.Dense(512,activation='relu')(y)
    y = layers.Dense(256,activation='relu')(y)
    y = layers.Dense(128,activation='relu')(y)
    y = layers.Dense(64,activation='relu')(y)
    y = layers.Dense(32,activation='relu')(y)
    y = keras.Model(inputs=input_mhc,outputs=y)

    combined = layers.concatenate([x.output,y.output])
    z = layers.Dense(64,activation='relu')(combined)
#     z = layers.Dropout(0.2)(z)
    z = layers.Dense(32,activation='sigmoid')(z)
    z = layers.Dropout(0.6)(z)
    z = layers.Dense(1,activation='sigmoid')(z)

    model = keras.Model(inputs=[input_pep,input_mhc],outputs=z)
    return model

In [ ]:
ann_model = buildANN()
ann_model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    )

callback_val = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,restore_best_weights=False)
callback_train = keras.callbacks.EarlyStopping(monitor='loss',patience=2,restore_best_weights=False)

history = ann_model.fit(
    x=[input1_train,input2_train],   # feed a list into
    y=label_train,
    validation_data = ([input1_val,input2_val],label_val),
    batch_size=128,
    epochs=200,
    callbacks = [callback_val,callback_train]
    )

In [ ]:
# plotting validation and training loss against number of epochs
plt.plot()
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()

In [ ]:
# predicting and further evaluating binary classification accuracy
predictions2 = ann_model.predict([input1_val,input2_val])
exact_match2 = 0
for i in range(len(predictions)):
    if predictions2[i]>0.5 and label_val[i]>0.5:
        exact_match2+=1
    elif predictions2[i]<=0.5 and label_val[i]<=0.5:
        exact_match2+=1
    else:
        continue
print("Accuracy : ",exact_match2/len(predictions2))

In [ ]:
# Dumping model to pickle file
pickle.dump(ann_model, open('ann_model.pkl', 'wb'))